# Variational Quantum Classifier (VQC) Best Parameters

This notebook will explore the the options for the best parameters to use for the VQC on the Pima Indians Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [2]:
# imports
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import clear_output

import os
import time

# import data class
from utilities.dataset_utils import DiabetesData

# import metrics for evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# qiskit imports
# circuit transpiler
from qiskit import transpile

# algorithm
from qiskit_machine_learning.algorithms.classifiers import VQC

# feature map
from qiskit.circuit.library import z_feature_map
from qiskit.circuit.library import zz_feature_map

# ansatz
from qiskit.circuit.library import real_amplitudes
from qiskit.circuit.library import efficient_su2

# optimizer
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.optimizers import ADAM
from qiskit_machine_learning.optimizers import SLSQP

# simulator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

In [3]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [4]:
# setup backend simulator
backend = AerSimulator()
backend.set_options(max_parallel_threads=os.cpu_count(), method='automatic')

# sampler
sampler = Sampler.from_backend(backend)

In [5]:
# define list for different ansatze as well as reps
ansatze = [
    transpile(efficient_su2(num_qubits=dataset.get_num_features(), reps=3, entanglement='linear'), backend=backend, optimization_level=2),
    transpile(real_amplitudes(num_qubits=dataset.get_num_features(), reps=3, entanglement='linear'), backend=backend, optimization_level=2)
]

In [6]:
# list of different feature maps
feature_maps = [
    transpile(z_feature_map(feature_dimension=dataset.get_num_features(), reps=2, entanglement='linear'), backend=backend, optimization_level=2),
    transpile(zz_feature_map(feature_dimension=dataset.get_num_features(), reps=2, entanglement='linear'), backend=backend, optimization_level=2)
]

In [7]:
# callback graph
# init list to store objective function values
objective_func_vals = []

# larger size for graph
plt.rcParams["figure.figsize"] = (12, 6)

# callback function to plot objective function value (updates after each iteration)
def callback_graph(weights, obj_func_eval):
    #     = -   --==``clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    # plt.title("Objective Function Value")
    # plt.xlabel("Iteration")
    # plt.ylabel("Objective Function Value")
    # plt.plot(objective_func_vals)
    # plt.show()

### VQC Building

Below is a function that will be iteratively called with multiple different parameters for training the VQC, to find the most optimal combination

In [8]:
def build_vqc(ansatz, feature_map, optimizer):
    vqc =  VQC(
        sampler=sampler,
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=callback_graph
    )
    return vqc

Preprocessing data, with different options for train size, to test quantum training on different sizes, as some studies suggest that QML algorithms perform equivelent or better than their classical counterparts with smaller dataset sizes

In [9]:
# list for train sizes
train_sizes = [200, 300, 400, 500]

Below function find the best combination of parameters for the VQC, as well as different dataset sizes

In [10]:
def combo_builder(ansatze: list, feature_maps: list, optimizer, train_sizes: list, X_train, y_train, X_test, y_test):
    results = []
    combo = 0
    
    # loop over different training sizes
    for sample_size in train_sizes:
        # if the training set is larger than the sample size, slice it
        if len(X_train) > sample_size:
            X_train_sample = X_train[:sample_size]
            y_train_sample = y_train[:sample_size]
        else:
            X_train_sample = X_train
            y_train_sample = y_train
        for ansatz in ansatze:
            for feature_map in feature_maps:
                objective_func_vals = []
                combo += 1
                print(f"Combination: {combo}\nAnsatz: {ansatz.name}\nFeature Map: {feature_map.name}\nOptimizer: {type(optimizer).__name__}\nSample Size: {sample_size}")
                
                # try parameter combo
                test_vqc = build_vqc(ansatz, feature_map, optimizer)

                # time how long it takes to train
                start = time.time()

                # fit the model
                test_vqc = test_vqc.fit(X_train_sample, y_train_sample)

                end = time.time()
                elapsed = end - start
                
                # sleep to avoid crashing
                time.sleep(10)
                
                train_score = test_vqc.score(X_train_sample, y_train_sample)
                test_score = test_vqc.score(X_test, y_test)
                
                # sleep to avoid crashing
                time.sleep(10)
                
                y_pred = test_vqc.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)

                data = {
                    "combo_num": combo,
                    "ansatz": ansatz.name,
                    "feature_map": feature_map.name,
                    "optmizer": type(optimizer).__name__,
                    "training_samples": sample_size,      
                    "train_time": elapsed,
                    "train_score": train_score,
                    "test_score": test_score,
                    "accuracy": accuracy,
                    "obj_func_eval": objective_func_vals
                }
            
                results.append(data)
                print("===COMBINATION COMPLETION===")
                
                # sleep to avoid crashing
                time.sleep(10)
    return results
                

### Model Fit for Best Params

In [11]:
# get training and testing data
X_train, X_test, y_train, y_test = dataset.preprocess_data()

In [12]:
# for COBYLA
r = combo_builder(ansatze, feature_maps, COBYLA(maxiter=300), train_sizes, X_train, y_train, X_test, y_test)
print(r)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Combination: 1
Ansatz: EfficientSU2
Feature Map: ZFeatureMap
Optimizer: COBYLA
Sample Size: 200
===COMBINATION COMPLETION===


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Combination: 2
Ansatz: EfficientSU2
Feature Map: ZZFeatureMap
Optimizer: COBYLA
Sample Size: 200
===COMBINATION COMPLETION===


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

Combination: 14
Ansatz: EfficientSU2
Feature Map: ZZFeatureMap
Optimizer: COBYLA
Sample Size: 500
===COMBINATION COMPLETION===


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Combination: 15
Ansatz: RealAmplitudes
Feature Map: ZFeatureMap
Optimizer: COBYLA
Sample Size: 500
===COMBINATION COMPLETION===


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Combination: 16
Ansatz: RealAmplitudes
Feature Map: ZZFeatureMap
Optimizer: COBYLA
Sample Size: 500
===COMBINATION COMPLETION===
[{'combo_num': 1, 'ansatz': 'EfficientSU2', 'feature_map': 'ZFeatureMap', 'optmizer': 'COBYLA', 'training_samples': 200, 'train_time': 269.8153622150421, 'train_score': 0.79, 'test_score': 0.7316017316017316, 'accuracy': 0.7359307359307359, 'obj_func_eval': []}, {'combo_num': 2, 'ansatz': 'EfficientSU2', 'feature_map': 'ZZFeatureMap', 'optmizer': 'COBYLA', 'training_samples': 200, 'train_time': 311.84609150886536, 'train_score': 0.805, 'test_score': 0.7012987012987013, 'accuracy': 0.696969696969697, 'obj_func_eval': []}, {'combo_num': 3, 'ansatz': 'RealAmplitudes', 'feature_map': 'ZFeatureMap', 'optmizer': 'COBYLA', 'training_samples': 200, 'train_time': 224.12580037117004, 'train_score': 0.72, 'test_score': 0.7012987012987013, 'accuracy': 0.7056277056277056, 'obj_func_eval': []}, {'combo_num': 4, 'ansatz': 'RealAmplitudes', 'feature_map': 'ZZFeatureMap', 'op